---
# Xlm Roberta Toxicity Predictions
---
<img src='//1.x24.fr/a/y/evasion2.jpg' style='max-width:100vw'>
- Welcome to my notebook, hope you won't mind for my brievity - just need vacations - First is a quick pre encoded data import
- Wishing you happy covid holidays to everyone :)
---

### Fix TF to version 2.2
- Avoid memory Stacking Issue

In [ ]:
from time import time, sleep
begins = time()
import os
from IPython.display import clear_output
#No OOM with tensorflow 2.2 ( works like a charm in colab )
!pip uninstall -y tensorflow
!pip install tensorflow==2.2.0
!pip install transformers
clear_output()
#!pip freeze | grep tensorflow

### Generic Modules

In [ ]:
#%load_ext memory_profiler

import numpy as np
import datetime
import pickle
import gc
import re
import unidecode
from memory_profiler import memory_usage
import requests
import pickle
import threading

# keep 6 minutes out of 3 hours for generating output
endOfTimes = (3600 * 2.8)
rdSeed = 1984
p = print

class monitoring: 
    def __init__(self): 
      self._running = True
    def terminate(self): 
      self._running = False   
    def run(self,msg,secSleep): 
      while self._running:
        r = requests.post('https://1.x24.fr/a/kaggle.php',data=msg)
        sleep(secSleep)

''' envoyer des packets heartbeats au serveur de monitoring '''
backgroundThread=monitoring();
t=threading.Thread(target=backgroundThread.run,args=({'heartbeat':'kaggleNotebookXlmRoberta'},60));
t.daemon=True;
t.start();

lastexec = datetime.datetime.now().strftime("%Y%m%d-%H%M")
clear_output()
p('last run:', lastexec)

---
# Submitting Latest Cloud Computations
- ( same notebook without time or memory constrains )
---

In [ ]:
target='p77d4_xlmr_300fast_.tgz'
latestResults='http://1.x24.fr/a/jupyter/poc7/'
r=requests.get(latestResults+target,stream=True)

with open(target,'wb') as f:
    f.write(r.raw.read())       

os.system('tar xf '+target)

target=target.replace('.tgz','.pickle')
data = open(target, "rb")
preds = pickle.load(data)
data.close()
os.system('rm '+target)

if 'reducting Dimensions within dict':
  pk=preds.keys();
  for i in pk:
    if type(preds[i][0])==np.ndarray:
      preds[i]=[ji[0] for ji in preds[i]]

df=pd.DataFrame(preds)
means=df.mean(axis=1).values
    
pk=list(preds.keys())

sep=pd.DataFrame({'id':list(range(0,63812))})
#sep['toxic']=preds[pk[-1]];#last prediction
sep['toxic']=means
fn='submission.csv'
sep['id,toxic'.split(',')].to_csv(fn,index=False)

In [ ]:

''' PreLoads Pre Processed Data in order to buy us some time since the clock is ticking '''


def loadData(fn):
    os.system('tar xf ../input/jigsawcsv/' + fn + '.tgz')
    data = open(fn + '.pickle', "rb")
    ret = pickle.load(data)
    data.close()
    os.system('rm -f ' + fn + '.pickle')
    return ret


memGraph = []
toxics = loadData('toxics')
totRows = len(toxics)
print('preloaded rows encodings :', totRows, 'rows')

nbDataSplits = 32
nbMax = 3200000
indexes = range(0, 100000)
toxics = toxics[:nbMax]
toxics_ = np.array_split(toxics, nbDataSplits)

---
# Merge dataframes
- What happens before generating the preload data - remove duplicates, pre-encode content - step is skipped in order to buy us some time
---

In [ ]:
from transformers import TFAutoModel, AutoTokenizer


def ftpexists(x):
    return False


def extractionMots3(x):
    x = unidecode.unidecode(x)
    x = re.sub('\n+', '.', x)  # newline
    x = re.sub(r'\.+', '.', x)  # single separator
    x = re.sub(r'@[a-zA-Z0-9\_]+', '', x)  # strip usernames
    x = re.sub(
        r'[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}',
        '',
        x)  # strip ips
    x = re.sub(r':[a-zA-Z0-9\_]+', '', x)  # kindof
    # strip urls with negative lookahead :)
    x = re.sub(r'https?\S+(?=\s|$)', '', x)
    # suppression tags ouverture et fermeture ( conservant leur contenus
    # interne : code, texte mis en forme, etc .. )
    notags = re.sub('<[^<]+?>', '', x)
    noHTMLentities = re.sub('&[^;]{1,9};', '', notags)  # cleanup &amp; &gt &lt
    stripped = re.sub(
        r"[^a-zA-Z0-9',.!\?]+",
        ' ',
        noHTMLentities,
        flags=re.IGNORECASE)  # autres que caractères de base
    monospaced = re.sub(r'\s+', ' ', stripped)
    trimmed = monospaced.strip(' .')
    return trimmed


def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_masks=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )

    return np.array(enc_di['input_ids'])


if False & bool(
        'what happened at the previous step ? the Magic under the hood'):
    # jigsaw-multilingual-toxic-comment-classification
    import pandas as pd
    import re

    def countwords(i):
        return len(re.findall(r'\w+', i))
    # getFile('data/googleApiComments');
    plangs = []
    import glob
    x = glob.glob(
        '../input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-*-cleaned.csv')
    for i in x:
        m = re.match(r'.*google-([^\-]+)-cleaned\.csv', i, re.M | re.I)
        if m:
            lang = m.group(1)
            p4 = pd.read_csv(i)['comment_text,toxic'.split(',')]
            p4['lang'] = lang
            p4['f'] = i
            plangs += [p4]
            del(p4)
        else:
            p('no lang for ', i)
            assert(False)

    f = '../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv'
    getFile(f)
    p1 = pd.read_csv(f)['comment_text,toxic'.split(',')]
    p1['lang'] = 'en'
    p1['f'] = f
    plangs += [p1]
    f = '../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv'
    getFile(f)
    p2 = pd.read_csv(f)['comment_text,toxic'.split(',')]
    p2['lang'] = 'en'
    p2['f'] = f
    plangs += [p2]
    f = '../input/jigsaw-multilingual-toxic-comment-classification/validation.csv'
    getFile(f)
    p3 = pd.read_csv(f)['comment_text,toxic,lang'.split(',')]
    p3['f'] = f
    plangs += [p3]  # various languages
    f = '../input/jigsaw-multilingual-toxic-comment-classification/jigsaw_miltilingual_valid_translated.csv'
    getFile(f)
    p4 = pd.read_csv(f)  # ['comment_text,toxic'.split(',')];
    p4['f'] = f
    p5 = p4.copy()['translated,toxic'.split(',')]
    p5.rename({'translated': 'comment_text'}, axis=1, inplace=True)
    p5['lang'] = 'en'

    p4.drop(['translated', 'id'], axis=1, inplace=True)
    plangs += [p4]
    plangs += [p5]

    pMulti = pd.concat(plangs)
    del(plangs, p1, p2, p3, p4, p5)

    duplicateRowsDF = pMulti[pMulti.duplicated(['comment_text'])]
    p('duplicateRowsDF:', duplicateRowsDF.shape[0])  # 42271 doublons
    dupIndexs = list(duplicateRowsDF.index)
    pMulti.drop(dupIndexs, inplace=True)

    courtes = pMulti['comment_text'].str.len() < 10
    courtes2Remove = pMulti[courtes].index
    # 6282 phrases trop courtes mises de côtéload('pMulti2')
    p('courtes2Remove:', len(courtes2Remove))

    pMulti2 = pMulti
    # **** Cleanup input texts, spaces, accents
    texts = pMulti2['comment_text'].values
    a = time()
    texts = [extractionMots3(i) for i in texts]
    p('time:', time() - a)
    pMulti2['comment_text'] = texts

    duplicateRowsDF = pMulti2[pMulti2.duplicated(['comment_text'])]
    p('duplicateRowsDF:', duplicateRowsDF.shape[0])  # 49648
    dupIndexs = list(duplicateRowsDF.index)
    pMulti2.drop(dupIndexs, inplace=True)

    courtes = pMulti2['comment_text'].str.len() < 10
    courtes2Remove = pMulti2[courtes].index
    p('courtes2Remove:', len(courtes2Remove))  # 7844
    pMulti2.drop(courtes2Remove, inplace=True)
    p('total rows:', pMulti2.shape[0])
    pMulti2.reset_index(inplace=True)
    '''
    duplicateRowsDF: 57669
    courtes2Remove: 7790
    total rows: 3200751
    '''

if False & bool('préencoded DataFrame Comments'):
    mls = [192]  # ,192,512
    models = {
        'xlmroberta': 'jplu/tf-xlm-roberta-large',
        # 'xlnet':'xlnet-large-cased',
        # 'multiDistil':'distilbert-base-multilingual-cased',
        # 'bert':'bert-base-multilingual-cased',
    }
    for df in [pMulti2]:
        for mdln in models.keys():
            mdl = models[mdln]
            for ml in mls:
                tokenizer = AutoTokenizer.from_pretrained(mdl)
                key = df + '_encoded_' + mdln + '_' + str(ml)
                key2 = key + '_2'
                key3 = 'vf_' + key  # vf_pMulti2_encoded_xlmroberta_300
                if ftpexists(key):
                    p('skipping generation, takes lots of time to tokenize each sentences, allready generated : ', key)
                else:
                    p(key)
                    a = time()
                    globals()[key] = regular_encode(
                        globals()[df]['comment_text'].values, tokenizer, maxlen=ml)
                    p(df, mdln, 'time:', time() - a)
                    assert(len(globals()[key]) == globals()[df].shape[0])
                    # save(key)
                    del(globals()[key])
                    gc.collect()

                if ftpexists(key3):
                    pass
                else:
                    a = time()
                    vf = testDf['content'].values
                    vf = [extractionMots3(i) for i in vf]
                    globals()[key3] = regular_encode(vf, tokenizer, maxlen=ml)
                    p(key3, 'time:', time() - a)
                    # save(key3)
                    del(globals()[key3])
                    gc.collect()


---
# Generic functions and imports
---

In [ ]:
from operator import itemgetter
from sklearn.metrics import roc_auc_score
from tensorflow.keras import backend as K
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm.notebook import tqdm
from transformers import TFAutoModel, AutoTokenizer
import transformers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input
import tensorflow.keras.backend as K
import tensorflow as tf
import pandas as pd
import sklearn.model_selection
import requests
import torch

def notification(x):
    r = requests.post('https://1.x24.fr/a/kaggle.php',data={0:x})

def seed_everything(seed=rdSeed):
    tf.random.set_seed = seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


tpu = False
shallRestart = False
seed_everything(rdSeed)
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 1
clear_output()

def getModel(mdlname,mdl,inputLen,loss='binary_crossentropy',nbOut=1,dense=0,freeze=0):  
  import tensorflow as tf
  @tf.function( experimental_relax_shapes=True )
  def f(x):
    return x
  from tensorflow.keras.layers import Dense, Input
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras.models import Model
  from transformers import TFAutoModel, AutoTokenizer
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D

  with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(mdl)#large is > base
    input_word_ids = Input(shape=(inputLen,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer_layer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    if dense>0:#      
      dense_layer = Dense(dense, activation='relu')(cls_token);
      out = Dense(nbOut, activation='sigmoid')(dense_layer)#32 nuances#bcp plus lourd à entrainer !
    else:
      out = Dense(nbOut, activation='sigmoid')(cls_token)#, dtype='float16'
    mdl = Model(inputs=input_word_ids, outputs=out)
    mdl._name=mdlname
    if type(freeze) is not int:
      for layer in mdl.layers:#Figer ceux du dessus
        if(type(layer)==freeze):
          p(layer,'not trainable')
          layer.trainable = False 

    mdl.compile(tf.keras.optimizers.Adam(lr=1e-5),loss=loss, metrics=['accuracy'])   
    #categorical_crossentropy,focal loss needs,tf.keras.metrics.AUC()
    #mdl.compile(tf.keras.optimizers.Adadelta(lr=1e-5,rho=0.95,epsilon=1e-07), loss='binary_crossentropy', metrics=['accuracy','auc'])#auroc,tf.keras.metrics.BinaryAccuracy() ,produit de très mauvais résultats ...
    #model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', auroc]

  return mdl

---
# Tpu inline
---

In [ ]:
if bool('get tpu'):
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
  except ValueError:
    tpu = None

  if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
  else:
    strategy = tf.distribute.get_strategy()     
     
clear_output()
print("REPLICAS: ", strategy.num_replicas_in_sync)
print(datetime.datetime.now().strftime("%Y%m%d-%H%M"))
assert(strategy.num_replicas_in_sync==8)

---
# Training function
---

In [ ]:
def submitResults():
    valDf = loadData('vf_pMulti2_encoded_xlmroberta_192')
    predictions = model.predict(valDf, batch_size=bs)

    testDf = pd.DataFrame({'id': list(range(0, 63812))})
    testDf['toxic'] = predictions
    fn = 'submission.fromKernel.' + datetime.datetime.now().strftime("%Y%m%d-%H%M")
    testDf['id,toxic'.split(',')].to_csv(
        fn + '.csv', index=False)  # not rounded
    del(testDf, valDf, predictions)
    gc.collect()


def saveWeights():
    fn = 'modelWeights.h5'
    os.system('rm -f ' + fn)
    model.save_weights(fn)


def trainLoop2(mon='loss', minmax='min', skfSplits=50):
    ''' Faster Pour chaque modèle le split stratifié d'entrainement Simple '''
    global x, y, model, killed, memGraph
    splits = np.array_split(indexes, skfSplits)  # depend du TPU
    j = 0
    for split in splits:
        mem_usage = memory_usage()
        memGraph += mem_usage

        if int(mem_usage[0]) + (2200 + 500) > 12500:
            print('short on memory', mem_usage[0], 'aborting')
            killed = 1
            del(splits, split)
            return

            if False:
                del(model)
                gc.collect()
                mem_usage = memory_usage()
                memGraph += mem_usage
                print('model deleted', mem_usage[0])
                model = getModel(mdlname, mdl, ml)
                model.load_weights('modelWeights.h5')
                mem_usage = memory_usage()
                memGraph += mem_usage
                print('model reloaded with weights', mem_usage[0])

        print(j, end=',')  # datetime.datetime.now().strftime("%Y%m%d-%H%M"))
        if (time() - begins) > endOfTimes:
            print('break training due to missing time')
            killed = 1
            return
        j += 1
        # IndexError: index 360701 is out of bounds for axis 0 with size 320000
        # ValueError: The number of samples 106692 is not divisible by batch
        # size 80.
        model.fit(x, y, epochs=EPOCHS, batch_size=bs)
        del(x, y)
        gc.collect()


ml = 192
mdlname = 'xlmr192Large_fast'
mdl = 'jplu/tf-xlm-roberta-large'
model = getModel(mdlname, mdl, ml,freeze=transformers.modeling_tf_roberta.TFRobertaModel)

---
# Training Loop and result submission
- Memory usage keeps on getting higher until the notebook crashes ..
---

In [ ]:
nbPerSplit = 100000
bs = 80
nbSplits = 40
bs = 8 * 50
nbSplits = 100  # 2 min per 50000
killed = saved = iLoop = 0
indexes = range(0, nbPerSplit)
data={}


while iLoop < 90:

    print('iLoop:', iLoop)  # datetime.datetime.now().strftime("%Y%m%d-%H%M")

    for index in range(0, nbDataSplits):  # +1 ==> oob
        # datetime.datetime.now().strftime("%Y%m%d-%H%M"))
        if (time() - begins) > endOfTimes:
            print('break training due to missing time')
            iLoop = 99999
            break

        mem_usage = memory_usage()
        memGraph += mem_usage
        if int(mem_usage[0]) + (2200 + 500) > 12500:
            print('short on memory', mem_usage[0], 'aborting')
            iLoop = 99999
            break

        y = toxics_[index].round().astype('bool')
        #load once, now we have more memory left
        if index not in data.keys():
            data[index] = np.array(loadData('pMulti2_encoded_xlmroberta_192_' + str(index)))
            
        a = time()
        model.fit(data[index], y, epochs=EPOCHS, batch_size=bs)
        print('training step ',index,'took:', round(time() - a), 'seconds')

    iLoop += 1
p('nbTrains loops',iLoop*index)

---
# Submiting results
---

In [ ]:
def plot(df, i='x', j='y', rotate=False, fn=False, title=False,width=6,height=6):
    if type(df)==dict:    
        df=pd.DataFrame.from_dict({i:list(df.keys()),j:list(df.values())}).sort_values(by=i,ascending=True)

    x, y, fn2, fig, ax = pltinit(df, i, j, title,height,width)
    if(fn==False):
        fn=fn2
    # bestCorrelationsKeys.keys():
    plt.plot(x, y)
    if rotate:
        plt.xticks(rotation=rotate
                   
plot(dict(memGraph),'train','memory',title='Memory usage per train')                   
#p(memGraph)#todo : plot memory usage
submitResults()
saveWeights()

In [ ]:
notification('kaggle xml roberta notebook execution completed in '+str(round(time()-begins))+' sec')    
p('exec time:',round(time()-begins),'sec')
backgroundThread.terminate()

---
<a target=1 href='//alpow.fr#o:kaggle'><img align=left src='https://alpow.fr/y/alpowPinkGradientNoFr.webp' style='max-width:50%;padding:0 20px 0'/></a> #Made with Love ♡ in the Alps by <a target=1 href='//alpow.fr#o:kaggle'>Alpow</a> ♡ <br>

- Hope you enjoyed how quick and simple was this notebook :)
<br><br><br><br><br>
<img src='//1.x24.fr/a/y/evasion2.jpg' style='max-width:100vw'>

 --- 